In [1]:
'''Prepare the environment and import the necessary modules'''

import pandas as pd
import os
import sys
from dotenv import load_dotenv
import ssl

sys.path.append('..')
sys.path.append('../..')
sys.path.append('../../..')
ssl._create_default_https_context = ssl._create_stdlib_context
load_dotenv()

# Must restart the kernel if any of the following imported modules are changed
from helpers.focusmate import fm_raw_sessions_to_df
from api.helpers.focusmate import fetch_all_focusmate_sessions, \
  fetch_focusmate_profile
from api.helpers.request import get_access_token
from api.test.sample_data.sessions import sample_sessions

fm_api_profile_endpoint = os.getenv("NEXT_PUBLIC_FM_API_PROFILE_ENDPOINT")
fm_api_sessions_endpoint = os.getenv("NEXT_PUBLIC_FM_API_SESSIONS_ENDPOINT")
session_id = ""
local_timezone = "America/New_York"

In [2]:
'''Get the sessions DataFrame'''

access_token = get_access_token(session_id)

profile_data = fetch_focusmate_profile(
        fm_api_profile_endpoint, access_token).get("user")
local_timezone: str = profile_data.get("timeZone")

# Option A: use all sessions
member_since: str = profile_data.get("memberSince")
fm_raw_sessions = await fetch_all_focusmate_sessions(
    fm_api_sessions_endpoint, access_token, member_since)

# Option B: use sample sessions
# sessions_data = sample_sessions

fm_sessions_df = fm_raw_sessions_to_df(fm_raw_sessions, local_timezone)
sessions = fm_sessions_df[fm_sessions_df['completed'] == True].copy()

sessions_copy = sessions.copy()

In [3]:
'''Playground space'''


,session_id,duration,start_time,requested_at,joined_at,completed,session_title,partner_id,start_date_str
0,8de6e574-1f70-4cc3-8060-04633dacf165,3000000,2021-11-03 21:45:00,2021-11-03 20:11:46.057499,2021-11-03 21:44:49.242829,True,,818119b2-7104-4f9f-880b-9c596d263934,Nov 21
1,b84323ad-1db0-41b2-8cfa-964703872574,3000000,2021-11-03 20:30:00,2021-11-03 20:11:14.318313,2021-11-03 20:29:57.785920,True,,3090687b-2f53-4bbf-b1f3-50462ecf9073,Nov 21
2,4458b55f-c68f-4b71-abe5-f086b7d3586f,3000000,2021-10-23 22:00:00,2021-10-23 21:51:43.130714,2021-10-23 22:00:04.025264,True,,d9ad67c3-2200-48b1-82bb-e5048d620f2e,Oct 21
3,d5d17aa4-594d-46ec-9df5-81f30e5715f6,3000000,2021-10-23 21:00:00,2021-10-23 20:57:48.443387,2021-10-23 20:59:58.820775,True,,395ede9e-ec01-4c4a-be11-38be4bdf664b,Oct 21
4,9b990cb4-e5e0-4ccd-b36a-b0c59ee3bbfd,3000000,2021-10-10 15:15:00,2021-10-10 15:05:18.891649,2021-10-10 15:07:30.672776,True,,17caeb70-f0e5-4294-8b6e-f4b5f51b056b,Oct 21
